In [1]:
import pandas as pd
import numpy as np
import math
import random

In [2]:
filename = "sap_storing_data_hu_project.csv" 
n = sum(1 for line in open(filename, encoding='utf8'))-1  
s = n//10
skip = sorted(random.sample(range(1, n+1), n-s))
df_not_cleaned = pd.read_csv(filename, skiprows=skip, low_memory=False)
#verwijder alle kolommen die niet van gebruik zijn:
df = df_not_cleaned.drop(columns=['stm_sap_mon_meld_ddt', 'stm_mon_begin_ddt', 'stm_mon_toelichting_trdl', 'stm_oh_pg_mld', 'stm_geo_mld_uit_functiepl', 'stm_scenario_mon', 'stm_status_melding_sap', 'stm_mon_nr_status_omschr', 'stm_mon_nr__statuscode', 'stm_mon_nr_status_wijzdd', 'stm_aanntpl_ddt', 'stm_geo_gst_uit_functiepl', 'stm_objectdl_code_gst', 'stm_objectdl_groep_gst', 'stm_progfh_in_ddt', 'stm_progfh_in_invoer_ddt', 'stm_progfh_gw_ddt', 'stm_progfh_gw_lwd_ddt', 'stm_progfh_hz', 'stm_veroorz_groep', 'stm_veroorz_code', 'stm_progfh_gw_duur', 'stm_progfh_gw_lwd_tijd', 'stm_progfh_gw_lwd_datum', 'stm_progfh_gw_datum', 'stm_arbeid', 'stm_functiepl_gst', 'stm_functiepl_mld', 'stm_mon_begintijd', 'stm_mon_begindatum', 'stm_aangelegd_tijd', 'stm_aangelegd_dd', 'stm_dir_betrok_tr', 'stm_evb', 'stm_mon_vhdsincident', 'stm_mon_eind_ddt', 'stm_afspr_aanvangddt', 'stm_effect', 'stm_veroorz_tekst_kort', 'stm_projnr', 'stm_wbi_nummer', 'stm_status_gebr', 'stm_standplaats', 'stm_actie', 'stm_hinderwaarde', 'stm_rec_toegev_ddt', 'stm_afsluit_tijd', 'stm_afsluit_dd', 'stm_afsluit_ddt', 'stm_akkoord_melding_jn', 'stm_fact_jn', 'stm_status_sapnaarmon', 'stm_akkoord_mon_toewijz', 'stm_controle_dd', 'stm_mon_eind_tijd', 'stm_mon_eind_datum', 'stm_reactie_duur', 'stm_afspr_aanvangtijd', 'stm_afspr_aanvangdd', 'stm_historie_toelichting', 'stm_schade_verhaalb_jn', 'stm_x_bron_arch_dt', 'stm_x_bron_bestandsnaam', 'stm_x_bron_publ_dt', 'stm_rapportage_jaar', 'stm_rapportage_maand', 'stm_sorteerveld', 'stm_afspr_func_hersteltijd', 'stm_afspr_func_hersteldd', 'stm_relatiervo', 'stm_relatiervo_volgend', 'stm_relatiervo_vorig', 'stm_schade_statusdatum', 'stm_schade_status_ga', 'stm_schadenr', 'stm_x_start_sessie_dt', 'stm_x_bk', 'stm_x_run_id', 'stm_x_actueel_ind', 'stm_x_vervallen_ind'])
print("Done")

Done


# Data cleaning:


De te bepalen welke variabelen het meest van toepassing zijn om te gebruiken kijken we naar welke
kolommen informatie geven over de tijd van de melding, monteur. Tijd wanneer de melding word opgelost.
Wat voor melding het is en info over de melding/storing, en de locatie. Zo zijn de volgende kolommen
het meest van toepassing:

Over soort melding
- stm_sap_meldtekst (melding omschrijving)
- stm_geo_mld (melding code) - nodig
- stm_oorz_code (oorzaak code) - nodig
- stm_prioriteit (prioriteit)

Over tijd:
- stm_sap_melddatum (datump melding) - nodig
- stm_sap_meldtijd (tijdstip melding) - nodig
- stm_aanngeb_dd (datum aannemer gebeld)
- stm_aanngeb_tijd (tijd aannemer gebeld)
- stm_aanntpl_dd (datum aannemer op plek)
- stm_aanntpl_tijd (tijd aannemer op plek)

- stm_fh_dd (Datum van functie herstel) - nodig
- stm_fh_tijd (Tijdstip van functieherstel) - nodig
- stm_sap_storeinddatum (Datum einde storing) - nodig
- stm_sap_storeindtijd (Tijdstip einde storing) - nodig
- stm_fh_duur (Duur van functiehersteltijd) - nodig

Over equipment:
- stm_equipm_nr_mld (equipment)
- stm_equipm_omschr_mld (equipment omschrijving)

Over locatie
- stm_km_van_mld (locatie van)
- stm_km_tot_mld (locatie tot)
- stm_vl_post (VL post) - nodig
- stm_contractgeb_mld(locatie melding) - nodig
- stm_pplg_van (PPLG van locatie)
- stm_pplg_naar (PPLG tot locatie)
- stm_dstrglp_van (Dienstregelpunt van locatie)
- stm_dstrglp_naar (Dienstregelpunt tot locatie)

Over prognose:
- stm_progfh_in_datum (Datum prognose)
- stm_progfh_in_tijd (Tijdstip FHT prognose)
- stm_progfh_in_invoer_dat (Invoer datum prognose)
- stm_progfh_in_invoer_tijd (Invoer tijdstip FHT prognose)
- stm_progfh_in_duur (Duur FHT)



In [3]:
#deze functie verwijdert alle rijen waar geen value op de gegeven kolommen zitten
def drop_rows_with_null(lst, df):
    dfisnull = df.isnull()[lst]
    matches = dfisnull[(dfisnull).any(axis=1)]
    return df.drop(matches.index)

In [4]:
# Verwijder de rijen die geen value bevatten in de benodigde kolommen
df_filtered = drop_rows_with_null(['stm_geo_mld', 'stm_sap_melddatum', 'stm_oorz_code', 'stm_sap_meldtijd',
                          'stm_fh_dd', 'stm_fh_tijd', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd',
                          'stm_fh_duur', 'stm_vl_post', 'stm_contractgeb_mld'], df)
print(len(df))
print(len(df_filtered))
df_filtered

89856
13345


,Unnamed: 0,#stm_sap_meldnr,stm_mon_nr,stm_vl_post,stm_sap_meld_ddt,stm_sap_meldtekst_lang,stm_sap_meldtekst,stm_geo_mld,stm_equipm_nr_mld,stm_equipm_soort_mld,...,stm_fh_dd,stm_fh_tijd,stm_fh_duur,stm_sap_storeinddatum,stm_sap_storeindtijd,stm_oorz_tekst_kort,stm_pplg_van,stm_pplg_naar,stm_dstrglp_van,stm_dstrglp_naar
19,208,50053591,39.0,EHV,20/01/2006 14:32:00,Logboeknr RBV : 39 Tijd: 1432 VL-Post: ...,Logboeknr RBV : 39 Tijd: 1432 VL-P,515.0,10527838.0,WISSEL,...,20/01/2006,14:47:00,15.0,23/01/2006,11:39:00,Geen SMC-melding.,NaN,NaN,Vga,NaN
29,253,50053664,103.0,ZL,26/01/2006 16:45:00,Logboeknr RBV : 103 Tijd: 1645 VL-Post: ...,Logboeknr RBV : 103 Tijd: 1645 VL-P,543.0,10642941.0,OVERWEG,...,26/01/2006,17:00:00,15.0,26/01/2006,17:00:00,vergeten de knop OPENEN te bedienen,NaN,NaN,Hgl,NaN
31,267,50053681,130.0,AH,29/01/2006 18:31:00,Logboeknr RBV : 130 Tijd: 1831 VL-Post: ...,Logboeknr RBV : 130 Tijd: 1831 VL-P,34.0,10211518.0,OVERWEG,...,29/01/2006,19:19:00,48.0,29/01/2006,19:19:00,vast zittende koolborstels,NaN,NaN,Rh,Vp
34,301,50053721,61.0,GN,30/01/2006 07:40:00,Logboeknr RBV : 61 Tijd: 0740 VL-Post: ...,Logboeknr RBV : 61 Tijd: 0740 VL-P,549.0,10221197.0,SEIN,...,30/01/2006,07:41:00,1.0,30/01/2006,07:41:00,extra vertraagd,NaN,NaN,Gn,NaN
53,533,50054073,122.0,RTD,23/02/2006 13:10:00,Logboeknr RBV : 122 Tijd: 1310 VL-Post: ...,Logboeknr RBV : 122 Tijd: 1310 VL-P,555.0,10038169.0,WISSELSTEL,...,23/02/2006,13:10:00,0.0,23/02/2006,13:10:00,BOIO,NaN,NaN,Rtd,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89786,908071,99002816,71.0,KFH,08/04/2006 10:04:00,Psein 830 is rood gedoofd.,Psein 830 is rood gedoofd.,950.0,10688510.0,SEIN,...,08/04/2006,16:05:00,359.0,08/04/2006,16:05:00,doorbranden,PS,NaN,RSCWA,NaN
89800,908161,99002911,282704.0,GVC,11/03/2008 20:19:00,Dt:tobs wiss 11b/13,Dt:tobs wiss 11b/13,112.0,10028290.0,DETGRSSSL,...,11/03/2008,21:10:00,49.0,11/03/2008,21:31:00,losse wisselverwarmingbuis maakte sluiti,DT,NaN,DT,NaN
89809,908236,99003005,974425.0,RTD,21/10/2010 19:42:00,"Dtz-Dt : Km 70.3-69.8, rijplaten waaien omhoo...","Dtz-Dt : Km 70.3-69.8, rijplaten waaien",112.0,10107602.0,BAANLICHAA,...,21/10/2010,20:59:00,77.0,21/10/2010,21:00:00,NaN,SDM,NaN,SDM,DTZ
89833,908413,99003167,1994752.0,RTD,09/05/2013 20:33:00,Brd : [2+] Brandmelding Station 2 sprinklerpo...,Brd : [2+] Brandmelding Station 2 sprink,116.0,10590754.0,TUNCONSTR,...,09/05/2013,21:11:00,37.0,09/05/2013,21:30:00,NaN,KFHAZ,NaN,KFHAZ,KFHAZ


In [5]:
#uitgeschreven lijsten zodat ik ze niet steeds hoef over te typen

# melding
df[['stm_sap_meldtekst', 'stm_geo_mld', 'stm_oorz_code', 'stm_prioriteit']]
# tijd
df[['stm_sap_melddatum', 'stm_sap_meldtijd', 'stm_aanngeb_dd', 'stm_aanngeb_tijd', 'stm_aanntpl_dd', 'stm_aanntpl_tijd']]
df[['stm_fh_dd', 'stm_fh_tijd', 'stm_fh_duur', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd']]
# equipment
df[['stm_equipm_omschr_mld', 'stm_equipm_nr_mld']]
# locatie
df[['stm_km_van_mld', 'stm_km_tot_mld', 'stm_vl_post', 'stm_contractgeb_mld', 'stm_pplg_van', 'stm_pplg_naar', 'stm_dstrglp_van', 'stm_dstrglp_naar']]
# prognose
df[['stm_progfh_in_datum', 'stm_progfh_in_tijd', 'stm_progfh_in_invoer_dat', 'stm_progfh_in_invoer_tijd', 'stm_progfh_in_duur']]


,stm_progfh_in_datum,stm_progfh_in_tijd,stm_progfh_in_invoer_dat,stm_progfh_in_invoer_tijd,stm_progfh_in_duur
0,NaN,00:00:00,NaN,00:00:00,99999999.0
1,NaN,00:00:00,NaN,00:00:00,99999999.0
2,NaN,00:00:00,NaN,00:00:00,99999999.0
3,NaN,00:00:00,NaN,00:00:00,99999999.0
4,NaN,00:00:00,NaN,00:00:00,99999999.0
...,...,...,...,...,...
89851,10/05/2013,20:15:00,10/05/2013,20:15:37,31
89852,11/05/2013,07:48:00,11/05/2013,07:58:00,4
89853,13/05/2013,03:30:00,13/05/2013,03:31:05,180
89854,11/05/2013,08:25:00,11/05/2013,08:31:24,4


# Baseline
Alle problemen worden in 3 uur opgelost.

In [6]:
begin_eind_tijd_dirty = df[['stm_sap_melddatum', 'stm_sap_meldtijd', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd']].dropna()
# begin en eindtijd in dataframe gezet.

In [7]:
begin_eind_tijd_clean = begin_eind_tijd_dirty.loc[(begin_eind_tijd_dirty['stm_sap_meldtijd'] != '::') & (begin_eind_tijd_dirty['stm_sap_storeindtijd'] != '::')]

In [8]:
stm_sap_melddatum_tijd_object = begin_eind_tijd_clean['stm_sap_melddatum'] + ' ' + begin_eind_tijd_clean['stm_sap_meldtijd']
stm_sap_melddatum_tijd = pd.DataFrame(stm_sap_melddatum_tijd_object,
                   columns=['Begin_tijd_en_datum']) # datum en tijd samengevoegd voor het makkelijker rekenen

In [9]:
stm_sap_einddatum_tijd_object = begin_eind_tijd_clean['stm_sap_storeinddatum'] + ' ' + begin_eind_tijd_clean['stm_sap_storeindtijd']
stm_sap_einddatum_tijd = pd.DataFrame(stm_sap_einddatum_tijd_object,
                   columns=['Eind_tijd_en_datum']) # datum en tijd samengevoegd voor het makkelijker rekenen

In [10]:
result_begin_eind_tijd = pd.concat([begin_eind_tijd_clean, stm_sap_melddatum_tijd, stm_sap_einddatum_tijd],axis=1).dropna()
result_begin_eind_tijd # de samengevoegde waarde in de dataframe gezet

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd,Begin_tijd_en_datum,Eind_tijd_en_datum
0,02/01/2006,09:00:00,02/01/2006,09:00:00,02/01/2006 09:00:00,02/01/2006 09:00:00
1,04/01/2006,12:27:00,04/01/2006,12:27:00,04/01/2006 12:27:00,04/01/2006 12:27:00
2,05/01/2006,00:33:00,05/01/2006,01:02:00,05/01/2006 00:33:00,05/01/2006 01:02:00
3,06/01/2006,18:25:00,06/01/2006,18:35:00,06/01/2006 18:25:00,06/01/2006 18:35:00
4,08/01/2006,13:19:00,09/01/2006,12:09:00,08/01/2006 13:19:00,09/01/2006 12:09:00
...,...,...,...,...,...,...
89850,10/05/2013,14:00:00,10/05/2013,15:41:00,10/05/2013 14:00:00,10/05/2013 15:41:00
89851,10/05/2013,19:41:00,10/05/2013,20:15:00,10/05/2013 19:41:00,10/05/2013 20:15:00
89853,11/05/2013,07:55:00,13/05/2013,03:30:00,11/05/2013 07:55:00,13/05/2013 03:30:00
89854,11/05/2013,07:59:00,11/05/2013,08:25:00,11/05/2013 07:59:00,11/05/2013 08:25:00


In [11]:
result_begin_eind_tijd['Begin_tijd_en_datum'] = pd.to_datetime(result_begin_eind_tijd['Begin_tijd_en_datum'])
result_begin_eind_tijd['Eind_tijd_en_datum'] = pd.to_datetime(result_begin_eind_tijd['Eind_tijd_en_datum'])

In [12]:
result_begin_eind_tijd

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd,Begin_tijd_en_datum,Eind_tijd_en_datum
0,02/01/2006,09:00:00,02/01/2006,09:00:00,2006-02-01 09:00:00,2006-02-01 09:00:00
1,04/01/2006,12:27:00,04/01/2006,12:27:00,2006-04-01 12:27:00,2006-04-01 12:27:00
2,05/01/2006,00:33:00,05/01/2006,01:02:00,2006-05-01 00:33:00,2006-05-01 01:02:00
3,06/01/2006,18:25:00,06/01/2006,18:35:00,2006-06-01 18:25:00,2006-06-01 18:35:00
4,08/01/2006,13:19:00,09/01/2006,12:09:00,2006-08-01 13:19:00,2006-09-01 12:09:00
...,...,...,...,...,...,...
89850,10/05/2013,14:00:00,10/05/2013,15:41:00,2013-10-05 14:00:00,2013-10-05 15:41:00
89851,10/05/2013,19:41:00,10/05/2013,20:15:00,2013-10-05 19:41:00,2013-10-05 20:15:00
89853,11/05/2013,07:55:00,13/05/2013,03:30:00,2013-11-05 07:55:00,2013-05-13 03:30:00
89854,11/05/2013,07:59:00,11/05/2013,08:25:00,2013-11-05 07:59:00,2013-11-05 08:25:00


In [13]:
fix_duur = result_begin_eind_tijd['Eind_tijd_en_datum'] - result_begin_eind_tijd['Begin_tijd_en_datum']

In [14]:
Duur_fix = pd.DataFrame(fix_duur,
                   columns=['Duur fix'])

In [15]:
Final_begin_eind_tijd = pd.concat([result_begin_eind_tijd, Duur_fix],axis=1).dropna()
Final_begin_eind_tijd

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd,Begin_tijd_en_datum,Eind_tijd_en_datum,Duur fix
0,02/01/2006,09:00:00,02/01/2006,09:00:00,2006-02-01 09:00:00,2006-02-01 09:00:00,0 days 00:00:00
1,04/01/2006,12:27:00,04/01/2006,12:27:00,2006-04-01 12:27:00,2006-04-01 12:27:00,0 days 00:00:00
2,05/01/2006,00:33:00,05/01/2006,01:02:00,2006-05-01 00:33:00,2006-05-01 01:02:00,0 days 00:29:00
3,06/01/2006,18:25:00,06/01/2006,18:35:00,2006-06-01 18:25:00,2006-06-01 18:35:00,0 days 00:10:00
4,08/01/2006,13:19:00,09/01/2006,12:09:00,2006-08-01 13:19:00,2006-09-01 12:09:00,30 days 22:50:00
...,...,...,...,...,...,...,...
89850,10/05/2013,14:00:00,10/05/2013,15:41:00,2013-10-05 14:00:00,2013-10-05 15:41:00,0 days 01:41:00
89851,10/05/2013,19:41:00,10/05/2013,20:15:00,2013-10-05 19:41:00,2013-10-05 20:15:00,0 days 00:34:00
89853,11/05/2013,07:55:00,13/05/2013,03:30:00,2013-11-05 07:55:00,2013-05-13 03:30:00,-177 days +19:35:00
89854,11/05/2013,07:59:00,11/05/2013,08:25:00,2013-11-05 07:59:00,2013-11-05 08:25:00,0 days 00:26:00


In [17]:
Final_begin_eind_tijd.loc[(Final_begin_eind_tijd['Duur fix'] > '0 days 00:00:00') & (Final_begin_eind_tijd['Duur fix'] < '0 days 03:00:00')]

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd,Begin_tijd_en_datum,Eind_tijd_en_datum,Duur fix
2,05/01/2006,00:33:00,05/01/2006,01:02:00,2006-05-01 00:33:00,2006-05-01 01:02:00,0 days 00:29:00
3,06/01/2006,18:25:00,06/01/2006,18:35:00,2006-06-01 18:25:00,2006-06-01 18:35:00,0 days 00:10:00
6,10/01/2006,22:25:00,10/01/2006,23:10:00,2006-10-01 22:25:00,2006-10-01 23:10:00,0 days 00:45:00
7,12/01/2006,05:30:00,12/01/2006,05:50:00,2006-12-01 05:30:00,2006-12-01 05:50:00,0 days 00:20:00
8,12/01/2006,16:25:00,12/01/2006,17:25:00,2006-12-01 16:25:00,2006-12-01 17:25:00,0 days 01:00:00
...,...,...,...,...,...,...,...
89849,10/05/2013,13:21:00,10/05/2013,13:27:00,2013-10-05 13:21:00,2013-10-05 13:27:00,0 days 00:06:00
89850,10/05/2013,14:00:00,10/05/2013,15:41:00,2013-10-05 14:00:00,2013-10-05 15:41:00,0 days 01:41:00
89851,10/05/2013,19:41:00,10/05/2013,20:15:00,2013-10-05 19:41:00,2013-10-05 20:15:00,0 days 00:34:00
89854,11/05/2013,07:59:00,11/05/2013,08:25:00,2013-11-05 07:59:00,2013-11-05 08:25:00,0 days 00:26:00
